# Stochastic Facility Location Model with Production Optimization

## Part a

### Decision Variables

1. **First-Stage Decisions** (made before scenarios are revealed):
   - $( y_j )$: Binary variable indicating whether facility \( $j$ \) is built (1 if built, 0 otherwise).

2. **Second-Stage Decisions** (adaptive to each scenario \( $s$ \)):
   - $( x_{js})$: Quantity produced at facility \( $j$ \) in scenario \( $s$ \).
   - $( z_{ijs} )$: Quantity transported from facility \( $j$ \) to customer zone \( $i$ \) in scenario \( $s$ \).
   - $( d_{is} )$: Quantity of demand met in customer zone \( $i$ \) in scenario \( $s$ \).

### Objective Function

The goal is to **minimize the total expected costs**, which include:
   - Facility construction costs
   - Expected production costs across scenarios
   - Expected transportation costs across scenarios
   - Expected costs of unmet demand across scenarios

The objective function is:

$$
\min \sum_{j=1}^{20} c_j y_j + \frac{1}{500} \sum_{s=1}^{500} \left( \sum_{j=1}^{20} q_{js} x_{js} + \sum_{i=1}^{50} \sum_{j=1}^{20} t_{ij} z_{ijs} + \sum_{i=1}^{50} u_i \left( D_{is} - d_{is} \right) \right)
$$

where \( $D_{is} - d_{is}$\) represents the unmet demand in zone \( $i$ \) for scenario \( $s$ \).

### Constraints

1. **Facility Capacity Constraints**: The quantity produced at each facility \( $j$ \) in each scenario \( $s$ \) should not exceed the facility's capacity and must be zero if the facility is not built:

   $$
   x_{js} \leq C_j y_j, \quad \forall j = 1, \dots, 20, \; s = 1, \dots, 500
   $$

2. **Demand Fulfillment Constraints**: The quantity of demand met \( $d_{is}$ \) should not exceed the total quantity transported to each customer zone \( $i$ \) from all facilities in each scenario \( $s$ \) and should not exceed customer demand either:

   $$
   d_{is} \leq \sum_{j=1}^{20} z_{ijs}, \quad \forall i = 1, \dots, 50, \; s = 1, \dots, 500
   $$

   $$
   d_{is} \leq D_{is}, \quad \forall i = 1, \dots, 50, \; s = 1, \dots, 500
   $$

3. **Flow Constraints**: The quantity transported from each facility \( $j$ \) to customer zones \( $i$ \) in each scenario \( $s$ \) cannot exceed the quantity produced at that facility:

   $$
   \sum_{i=1}^{50} z_{ijs} \leq x_{js}, \quad \forall j = 1, \dots, 20, \; s = 1, \dots, 500
   $$

4. **Non-negativity Constraints**: All quantities (produced, transported, and unmet demand) should be non-negative:

   $$
   x_{js} \geq 0, \; z_{ijs} \geq 0, \; d_{is} \geq 0, \quad \forall j = 1, \dots, 20, \; i = 1, \dots, 50, \; s = 1, \dots, 500
   $$

5. **Binary Constraints**: The first-stage variable indicating facility construction is binary:

   $$
   y_j \in \left\{ 0, 1 \right\}, \quad \forall j = 1, \dots, 20
   $$


## Part b / Part c

In [ ]:
using JuMP, Gurobi, CSV, DataFrames, Statistics

facilities = CSV.read("data/facilities.csv", DataFrame)
production = CSV.read("data/production.csv", DataFrame)
transportation = CSV.read("data/transportation.csv", DataFrame)
zones = CSV.read("data/zones.csv", DataFrame)
demand = CSV.read("data/demand.csv", DataFrame)

num_facilities = 20
num_zones = 50
num_scenarios = 500

c_j = facilities[:, 2]
C_j = facilities[:, 3]
q_js = Matrix(production)[:,2:end]
t_ij = Matrix(transportation)[:,2:end]
u_i = zones[:, 2]
D_is = Matrix(demand)[:, 2:end]

D_i_avg = mean(D_is, dims=2)
q_j_avg = mean(q_js, dims=2)

function solve_stochastic_model()
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)
    set_optimizer_attribute(model, "TimeLimit", 600)
    
    @variable(model, y[1:num_facilities], Bin)
    @variable(model, x[1:num_facilities, 1:num_scenarios] >= 0)
    @variable(model, z[1:num_zones, 1:num_facilities, 1:num_scenarios] >= 0)
    @variable(model, d[1:num_zones, 1:num_scenarios] >= 0)

    @objective(model, Min,
        sum(c_j[j] * y[j] for j in 1:num_facilities) +
        (1 / num_scenarios) * sum(
            sum(q_js[j, s] * x[j, s] for j in 1:num_facilities) +
            sum(t_ij[i, j] * z[i, j, s] for i in 1:num_zones, j in 1:num_facilities) +
            sum(u_i[i] * (D_is[i, s] - d[i, s]) for i in 1:num_zones)
            for s in 1:num_scenarios
        )
    )

    @constraint(model, [j=1:num_facilities, s=1:num_scenarios], x[j, s] <= C_j[j] * y[j])
    @constraint(model, [i=1:num_zones, s=1:num_scenarios], 
        d[i, s] <= sum(z[i, j, s] for j in 1:num_facilities)
    )
    @constraint(model, [i=1:num_zones, s=1:num_scenarios],
        d[i, s] <= D_is[i, s]
    )
    @constraint(model, [j=1:num_facilities, s=1:num_scenarios],
        sum(z[i, j, s] for i in 1:num_zones) <= x[j, s]
    )

    optimize!(model)
    return objective_value(model)
end

function solve_expected_value_model()
    model = Model(Gurobi.Optimizer)
    set_optimizer_attribute(model, "OutputFlag", 0)
    set_optimizer_attribute(model, "TimeLimit", 600)

    @variable(model, y[1:num_facilities], Bin)
    @variable(model, x[1:num_facilities] >= 0)
    @variable(model, z[1:num_zones, 1:num_facilities] >= 0)
    @variable(model, d[1:num_zones] >= 0)

    @objective(model, Min,
        sum(c_j[j] * y[j] for j in 1:num_facilities) +
        sum(q_j_avg[j] * x[j] for j in 1:num_facilities) +
        sum(t_ij[i, j] * z[i, j] for i in 1:num_zones, j in 1:num_facilities) +
        sum(u_i[i] * (D_i_avg[i] - d[i]) for i in 1:num_zones)
    )

    @constraint(model, [j=1:num_facilities], x[j] <= C_j[j] * y[j])
    @constraint(model, [i=1:num_zones], 
        d[i] <= sum(z[i, j] for j in 1:num_facilities)
    )
    @constraint(model, [i=1:num_zones],
        d[i] <= D_i_avg[i]
    )
    @constraint(model, [j=1:num_facilities],
        sum(z[i, j] for i in 1:num_zones) <= x[j]
    )

    optimize!(model)
    y_values = value.(y)  # Get the values of y variables
    return y_values
end

function solve_wait_and_see_model()
    ws_obj_value = 0.0
    for s in 1:num_scenarios
        model = Model(Gurobi.Optimizer)
        set_optimizer_attribute(model, "OutputFlag", 0)
        set_optimizer_attribute(model, "TimeLimit", 600)

        @variable(model, y[1:num_facilities], Bin)
        @variable(model, x[1:num_facilities] >= 0)
        @variable(model, z[1:num_zones, 1:num_facilities] >= 0)
        @variable(model, d[1:num_zones] >= 0)

        @objective(model, Min,
            sum(c_j[j] * y[j] for j in 1:num_facilities) +
            sum(q_js[j, s] * x[j] for j in 1:num_facilities) +
            sum(t_ij[i, j] * z[i, j] for i in 1:num_zones, j in 1:num_facilities) +
            sum(u_i[i] * (D_is[i, s] - d[i]) for i in 1:num_zones)
        )

        @constraint(model, [j=1:num_facilities], x[j] <= C_j[j] * y[j])
        @constraint(model, [i=1:num_zones], 
            d[i] <= sum(z[i, j] for j in 1:num_facilities)
        )
        @constraint(model, [i=1:num_zones],
            d[i] <= D_is[i, s]
        )
        @constraint(model, [j=1:num_facilities],
            sum(z[i, j] for i in 1:num_zones) <= x[j]
        )

        optimize!(model)
        ws_obj_value += objective_value(model) / num_scenarios
    end
    return ws_obj_value
end


stochastic_value = solve_stochastic_model()
wait_and_see_value = solve_wait_and_see_model()
y_ev = solve_expected_value_model()

function reevaluate_expected_value_solution(y_ev)
    reeval_obj_value = sum(c_j[j] * y_ev[j] for j in 1:num_facilities)

    for s in 1:num_scenarios
        model = Model(Gurobi.Optimizer)
        set_optimizer_attribute(model, "OutputFlag", 0)
        set_optimizer_attribute(model, "TimeLimit", 600)

        @variable(model, x[1:num_facilities] >= 0)
        @variable(model, z[1:num_zones, 1:num_facilities] >= 0)
        @variable(model, d[1:num_zones] >= 0)

        @objective(model, Min,
            sum(q_js[j, s] * x[j] for j in 1:num_facilities) +
            sum(t_ij[i, j] * z[i, j] for i in 1:num_zones, j in 1:num_facilities) +
            sum(u_i[i] * (D_is[i, s] - d[i]) for i in 1:num_zones)
        )

        @constraint(model, [j=1:num_facilities], x[j] <= C_j[j] * y_ev[j])
        @constraint(model, [i=1:num_zones], 
            d[i] <= sum(z[i, j] for j in 1:num_facilities)
        )
        @constraint(model, [i=1:num_zones],
            d[i] <= D_is[i, s]
        )
        @constraint(model, [j=1:num_facilities],
            sum(z[i, j] for i in 1:num_zones) <= x[j]
        )

        optimize!(model)
        reeval_obj_value += objective_value(model) / num_scenarios
    end
    return reeval_obj_value
end

# Reevaluate the expected cost of the Expected Value solution
expected_value = reevaluate_expected_value_solution(y_ev)

In [2]:
evpi = (stochastic_value-wait_and_see_value)/wait_and_see_value
vss = (expected_value - stochastic_value)/expected_value

println("Optimal Objective Value (Stochastic Solution): ", stochastic_value)
println("Expected Value Solution Objective: ", expected_value)
println("Wait-and-See Solution Objective: ", wait_and_see_value)
println("Expected Value of Perfect Information (EVPI): ", evpi)
println("Value of the Stochastic Solution (VSS): ", vss)

Optimal Objective Value (Stochastic Solution): 1206.7855862753831
Expected Value Solution Objective: 1427.0975961414001
Wait-and-See Solution Objective: 754.5529152395316
Expected Value of Perfect Information (EVPI): 0.5993385777222676
Value of the Stochastic Solution (VSS): 0.15437767568363836
